# MPIA Arxiv on Deck 2: Debugging notebook

In this notebook, I keep some first order commands for diagnostic of issues with papers.
Main definitions are taken from the main notebook.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

# Some useful definitions.
class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

We get the author list from the MPIA website

In [2]:
# !rm -f tmp_mpia_authors.yml

In [3]:
# Getting the list of authors can take sometimes (internet connection)
# Caching the MPIA author list to avoid running this line every time we restart the kernel.
import yaml
try:
    with open('tmp_mpia_authors.yml', 'r') as fin:
        mpia_authors = yaml.load(fin, yaml.BaseLoader)
    print("`mpia.get_mpia_mitarbeiter_list()`: restored from cache")
except FileNotFoundError:
    print("`mpia.get_mpia_mitarbeiter_list()`: cannot be restored from cache.")
    # get list from MPIA website
    # it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
    mpia_authors = mpia.get_mpia_mitarbeiter_list()
    with open('tmp_mpia_authors.yml', 'w') as fout:
        fout.write(yaml.dump(mpia_authors))

`mpia.get_mpia_mitarbeiter_list()`: restored from cache


We get the paper to debug

In [4]:
which = "2303.13732"
paper = get_paper_from_identifier(which)
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2303.13732-b31b1b.svg)](https://arxiv.org/abs/2303.13732) | **Thermal Emission from the Hot Jupiter WASP-103b in $J$ and $K_{\rm s}$ Bands**  |
|| Yaqing Shi, et al. |
|*Appeared on*| *2023-03-24*|
|*Comments*| **|
|**Abstract**| Hot Jupiters, particularly those with temperature higher than 2000\,K are thebest sample of planets that allow in-depth characterization of theiratmospheres. We present here a thermal emission study of the ultra hot JupiterWASP\mbox{-}103\,b observed in two secondary eclipses with CFHT/WIRCam in $J$and $K_{\rm s}$ bands. By means of high precision differential photometry, wedetermine eclipse depths in $J$ and $K_{\rm s}$ to an accuracy of 220 and270\,ppm, which are combined with the published HST/WFC3 and Spitzer data toretrieve a joint constraints on the properties of WASP-103\,b daysideatmosphere. We find that the atmosphere is best fit with a thermal inversionlayer included. The equilibrium chemistry retrieval indicates an enhanced C/O(1.35$^{+0.14}_{-0.17}$) and a super metallicity with[Fe/H]$=2.19^{+0.51}_{-0.63}$ composition. Given the near-solar metallicity ofWASP-103 of [Fe/H]=0.06, this planet seems to be $\sim$100 more abundant thanits host star. The free chemistry retrieval analysis yields a large abundanceof FeH, H$^{-}$, CO$_2$ and CH$_4$. Additional data of better accuracy fromfuture observations of JWST should provide better constraint of the atmosphericproperties of WASP-103b.|

In [5]:
# Check author list with their initials
normed_author_list = [mpia.get_initials(k) for k in paper['authors']]
normed_mpia_authors = [k[1] for k in mpia_authors]
hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
matches = [(hl, orig) for hl, orig in zip(hl_authors, paper['authors']) if 'mark' in hl]
if not matches:
    warnings.warn(AffiliationWarning("WARNING: This paper does not seem to have MPIA authors."))
    
paper['authors'] = hl_authors
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2303.13732-b31b1b.svg)](https://arxiv.org/abs/2303.13732) | **Thermal Emission from the Hot Jupiter WASP-103b in $J$ and $K_{\rm s}$ Bands**  |
|| Y. Shi, et al. -- incl., <mark>M. Zhai</mark>, <mark>R. v. Boekel</mark> |
|*Appeared on*| *2023-03-24*|
|*Comments*| **|
|**Abstract**| Hot Jupiters, particularly those with temperature higher than 2000\,K are thebest sample of planets that allow in-depth characterization of theiratmospheres. We present here a thermal emission study of the ultra hot JupiterWASP\mbox{-}103\,b observed in two secondary eclipses with CFHT/WIRCam in $J$and $K_{\rm s}$ bands. By means of high precision differential photometry, wedetermine eclipse depths in $J$ and $K_{\rm s}$ to an accuracy of 220 and270\,ppm, which are combined with the published HST/WFC3 and Spitzer data toretrieve a joint constraints on the properties of WASP-103\,b daysideatmosphere. We find that the atmosphere is best fit with a thermal inversionlayer included. The equilibrium chemistry retrieval indicates an enhanced C/O(1.35$^{+0.14}_{-0.17}$) and a super metallicity with[Fe/H]$=2.19^{+0.51}_{-0.63}$ composition. Given the near-solar metallicity ofWASP-103 of [Fe/H]=0.06, this planet seems to be $\sim$100 more abundant thanits host star. The free chemistry retrieval analysis yields a large abundanceof FeH, H$^{-}$, CO$_2$ and CH$_4$. Additional data of better accuracy fromfuture observations of JWST should provide better constraint of the atmosphericproperties of WASP-103b.|

We get the (TeX) source
* retrieve the tarball
* find the main tex file and parse it
* parse for affiliations (but debugging so we do not stop if not found)
* generate the the output markdown

In [6]:
def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

In [7]:
paper_id = f'{which:s}'
folder = f'tmp_{paper_id:s}'

if not os.path.isdir(folder):
    folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')

try:
    doc = latex.LatexDocument(folder, validation=validation)    
except AffiliationError as affilerror:
    msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
    print(msg)

# Hack because sometimes author parsing does not work well
if (len(doc.authors) != len(paper['authors'])):
    doc._authors = paper['authors']
else:
    # highlight authors (FIXME: doc.highlight_authors)
    # done on arxiv paper already
    doc._authors = highlight_authors_in_list(
        [mpia.get_initials(k) for k in doc.authors], 
        normed_mpia_authors, verbose=True)
if (doc.abstract) in (None, ''):
    doc._abstract = paper['abstract']

doc.comment = get_markdown_badge(paper_id) 
if paper['comments']:
    doc.comment += " _" + paper['comments'] + "_"

full_md = doc.generate_markdown_text()

full_md += get_markdown_qrcode(paper_id)

# replace citations
try:
    bibdata = latex_bib.LatexBib.from_doc(doc)
    full_md = latex_bib.replace_citations(full_md, bibdata)
except Exception as e:
    raise e

/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.13732/W103b_R2.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


Found 90 bibliographic references in tmp_2303.13732/W103b_R2.bbl.


In [8]:
paper['comments']

''

In [9]:
Markdown(full_md)

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ww}[1]{{\textcolor{black}{#1}}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# THERMAL EMISSION FROM THE HOT JUPITER WASP-103 b IN $J$ AND $K{\rm s}$ BANDS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.13732-b31b1b.svg)](https://arxiv.org/abs/2303.13732)

</div>
<div id="authors">

Y. Shi, et al. -- incl., <mark>M. Zhai</mark>, <mark>R. v. Boekel</mark>

</div>
<div id="abstract">

**Abstract:** Hot Jupiters, particularly those with temperature higher than 2000 K are the best sample of planets that allow in-depth characterization of their atmospheres. We present here a thermal emission study of the ultra hot Jupiter WASP $\mbox{-}$ 103 b observed in two secondary eclipses with CFHT/WIRCam in $J$ and $K_{\rm s}$ bands. By means of high precision differential photometry, we determine eclipse depths in $J$ and $K_{\rm s}$ to an accuracy of 220 and 270 ppm, which are combined with the published HST/WFC3 and Spitzer data to retrieve a joint constraints on the properties of WASP-103 b dayside atmosphere. We find that the atmosphere is best fit with a thermal inversion layer included. The equilibrium chemistry retrieval indicates an enhanced C/O (1.35 $^{+0.14}_{-0.17}$ ) and a super metallicity with [ Fe/H ] $=2.19^{+0.51}_{-0.63}$ composition. Given the near-solar metallicity of WASP-103 of [ Fe/H ] =0.06, this planet seems to be $\sim$ 100 more abundant than its host star. The free chemistry retrieval analysis yields a large abundance of FeH, H $^{-}$ , $CO_2$ and $CH_4$ . Additional data of better accuracy from future observations of JWST should provide better constraint of the atmospheric properties of WASP-103b.

</div>

<div id="div_fig1">

<img src="tmp_2303.13732/./pic/model_103b.png" alt="Fig7" width="100%"/>

**Figure 7. -** Observations and model spectra of dayside thermal emission of WASP-103b. Our $J$ and $K_{\rm s}$ data are presented by purple solid square, and other archival data are presented by black solid circle. The orange line shows the EQ model, while the violet line shows the FREE model. $\ww${The two dotted lines show the EQ and FREE models assuming isothermal. } (*Fig7*)

</div>
<div id="div_fig2">

<img src="tmp_2303.13732/./pic/rxb2_J.png" alt="Fig2" width="100%"/>

**Figure 2. -** The normalized rms$\times\beta^{2}$ distribution overlaid with contour maps for the various combinations of aperture diameter $D$ and the number of reference star group ($N_{\rm RSG}$) in the $J$ band. The minimum value of rms$\times\beta^{2}$ is reached with $D=34$ and $N_{\rm RSG}=5$. (*Fig2*)

</div>
<div id="div_fig3">

<img src="tmp_2303.13732/./pic/rms-beta.png" alt="Fig6" width="100%"/>

**Figure 6. -** RMS and noise factor $\beta$ of our residuals to the best-fit model for the various data sets in $J$ band.  (*Fig6*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.13732"></div>

In [10]:
print(doc.abstract)

Hot Jupiters, particularly those with temperature higher than 2000 K are the best sample of planets that allow in-depth characterization of their atmospheres. We present here a thermal emission study of the ultra hot Jupiter WASP $\mbox{-}$ 103 b observed in two secondary eclipses with CFHT/WIRCam in $J$ and $K_{\rm s}$ bands. By means of high precision differential photometry, we determine eclipse depths in $J$ and $K_{\rm s}$ to an accuracy of 220 and 270 ppm, which are combined with the published HST/WFC3 and Spitzer data to retrieve a joint constraints on the properties of WASP-103 b dayside atmosphere. We find that the atmosphere is best fit with a thermal inversion layer included. The equilibrium chemistry retrieval indicates an enhanced C/O (1.35 $^{+0.14}_{-0.17}$ ) and a super metallicity with [ Fe/H ] $=2.19^{+0.51}_{-0.63}$ composition. Given the near-solar metallicity of WASP-103 of [ Fe/H ] =0.06, this planet seems to be $\sim$ 100 more abundant than its host star. The fre

In [9]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [10]:
export_markdown_summary(full_md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.12101.md
    + _build/html/tmp_2303.12101/fig/nebulae_age_tracers_corner.png
    + _build/html/tmp_2303.12101/fig/overlap_rgb.png
    + _build/html/tmp_2303.12101/fig/catalogue_properties_2D_hist.png
